# Agent Demo

In this Demo we are going to show how to use the ReACT agent to answer questions about a rock band.

Goal of this code is to showcase use case of [Intelligent Agents](https://python.langchain.com/docs/modules/agents/) capable of reading DuckDuckGo search results to properly answer questions about different topics.

Python code bellow makes use of the following software:
 - [langchain](https://www.langchain.com) - a library for building language chains
 - [OpenAI ChatGPT](https://platform.openai.com/docs/introduction) - a language model API

The agent is a implementation makes use of the [ReAct Prompting](https://react-lm.github.io) algorithm. A ReAct prompt consists of few-shot task-solving trajectories, with human-written text reasoning traces and actions, as well as environment observations in response to actions. 

This notebook is also available on [Google Colab](https://colab.research.google.com/drive/1IivBK3VFz3NkXxWbr1AdMKbZEBB47083?usp=sharing)

In [62]:
# imports
import langchain
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [69]:
# globals
api_key = "?"
question = "What is the latest album released by Red Hot Chili Peppers?"  # example question, a good one :D

## Model loading

In this block model is loaded.

In [70]:
# load model 
model = ChatOpenAI(openai_api_key=api_key, temperature=0.0)

## LLM direct inference
This block shows how to use the pipeline to pass a question directly to the model and get a response. 
It can be observed the model understands and replies the question, however the answer is not up to date with the latest album released by given rock band. It can very well be the case the dataset used to train the model is not up to date.

In [71]:
prompt_template =  ChatPromptTemplate.from_template("{input}")
result = model(prompt_template.format_messages(input=question))
print(result.content)

The latest album released by Red Hot Chili Peppers is "The Getaway," which was released on June 17, 2016.


## ReAct Agent with LLM backend inference

This block shows how to use the pipeline to pass a question to the agent and get a response.

The agent will fork the dialog between two threads and start a problem solving dialog using the ReAct algorithm.

It will first understand the question, search the main concept in  DuckDuckGo, read the search results and then answer the question. 

In [72]:
# load agent with wikipedia
tools = load_tools(["ddg-search"], llm=model)
agent = initialize_agent(
    tools,
    model,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True
)

In [73]:
# langchain.debug = True
result = agent(question)['output']
# langchain.debug = False

In [74]:
print(f"ReACT result: {result}")

ReACT result: The latest album released by Red Hot Chili Peppers is "Return of the Dream Canteen".
